In [45]:
from neo4j import GraphDatabase
import json

In [46]:
with open('credentials.json') as json_file:
    credentials = json.load(json_file)

username = credentials['username']
pwd = credentials['password']

### NOTE ❣️

* BEFORE running this, still need to run `bin\neo4j console` to enable bolt on 127.0.0.1:7687
* When the queryyou wrote is wrong, the error will show connection or credential has problem, you don't really need to restart the server, after the query has been corrected, everything will be running fine.

#### Userful Links
* Results can be outputed: https://neo4j.com/docs/api/python-driver/current/results.html

In [47]:
driver = GraphDatabase.driver("bolt://localhost:7687", auth=(username, pwd))

In [113]:
def delete_all(tx):
    result = tx.run("""match (n) detach delete n""").single()
    if result is None:
        print('Removed All!')

def create_entity(tx, entity_id, entity_name, entity_properties):
    query = """CREATE ("""+entity_id+""": """+entity_name+entity_properties+""")"""
    result = tx.run(query)
    
def display_all(tx, query):
    results = tx.run(query)
    for record in results:
        print(record)
    return results.graph()

In [119]:
with driver.session() as session:
    session.write_transaction(delete_all)
    session.write_transaction(create_entity, entity_id='Alice', entity_name='Client',
        entity_properties = "{name:'Alice', ip: '1.1.1.1', shipping_address: 'a place', billing_address: 'a place'}")
    
    graph = session.read_transaction(display_all, query="MATCH (c:Client) RETURN c")

Removed All!
<Record c=<Node id=18 labels={'Client'} properties={'name': 'Alice', 'billing_address': 'a place', 'shipping_address': 'a place', 'ip': '1.1.1.1'}>>


In [114]:
def create_all(tx, query):
    result = tx.run(query)
    
    query = """
        // Clients
        CREATE (Alice:Client {name:'Alice', ip: '1.1.1.1', shipping_address: 'a place', billing_address: 'a place'})
        CREATE (Bob:Client {name:'Bob', ip: '1.1.1.2', shipping_address: 'b place', billing_address: 'b place'})
        CREATE (Cindy:Client {name:'Cindy', ip: '1.1.1.3', shipping_address: 'c place', billing_address: 'c place'})
        CREATE (Diana:Client {name:'Diana', ip: '1.1.1.4', shipping_address: 'd place', billing_address: 'd place'})
        CREATE (Emily:Client {name:'Emily', ip: '1.1.1.5', shipping_address: 'e place', billing_address: 'e place'})
        CREATE (Fiona:Client {name:'Fiona', ip: '1.1.1.6', shipping_address: 'f place', billing_address: 'f place'})

        // Products
        CREATE (prod1:Product {name: 'strawberry ice-cream', category: 'ice-cream', price: 6.9, unit: 'box'})
        CREATE (prod2:Product {name: 'mint ice-cream', category: 'ice-cream', price: 6.9, unit: 'box'})
        CREATE (prod3:Product {name: 'mango ice-cream', category: 'ice-cream', price: 6.9, unit: 'box'})
        CREATE (prod4:Product {name: 'cheesecake ice-cream', category: 'ice-cream', price: 7.9, unit: 'box'})
        CREATE (prod5:Product {name: 'orange', category: 'furit', unit: 'lb', price: 2.6, unit: 'box'})
        CREATE (prod6:Product {name: 'dragon fruit', category: 'furit', unit: 'lb', price: 4.8, unit: 'box'})
        CREATE (prod7:Product {name: 'kiwi', category: 'furit', unit: 'lb', price: 5.3, unit: 'box'})
        CREATE (prod8:Product {name: 'cherry', category: 'furit', unit: 'lb', price: 4.8, unit: 'box'})
        CREATE (prod9:Product {name: 'strawberry', category: 'furit', unit: 'lb', price: 3.9, unit: 'box'})

        // Orders
        CREATE (d1:Order {id:'d1', name:'d1', deliverdate:'20190410', status:'delivered'})
        CREATE (d2:Order {id:'d2', name:'d2', deliverdate:'20130708', status:'delivered'})
        CREATE (d3:Order {id:'d3', name:'d3', deliverdate:'20021201', status:'delivered'})
        CREATE (d4:Order {id:'d4', name:'d4', deliverdate:'20040612', status:'delivered'})
        CREATE (d5:Order {id:'d5', name:'d5', deliverdate:'20110801', status:'delivered'})
        CREATE (d6:Order {id:'d6', name:'d6',deliverdate:'20171212', status:'delivered'})

        // Link Clients, Orders and ProductsCREATE
        CREATE
            (Alice)-[:PLACED]->(d1)-[:CONTAINS {quantity:1}]->(prod1),
            (d1)-[:CONTAINS {quantity:2}]->(prod2),
            (Bob)-[:PLACED]->(d2)-[:CONTAINS {quantity:2}]->(prod1),
            (d2)-[:CONTAINS {quantity:6}]->(prod7),
            (Cindy)-[:PLACED]->(d3)-[:CONTAINS {quantity:1}]->(prod9),
            (Alice)-[:PLACED]->(d4)-[:CONTAINS {quantity:100}]->(prod4),
            (Alice)-[:PLACED]->(d5)-[:CONTAINS {quantity:10}]->(prod8),
            (Alice)-[:PLACED]->(d6)-[:CONTAINS {quantity:1}]->(prod7);
        """

In [115]:
with driver.session() as session:
    session.write_transaction(delete_all)
    session.write_transaction(create_all, query)
    
    graph = session.read_transaction(display_all, query="""MATCH (c:Client)-[:PLACED]-(o)-[:CONTAINS]->(p)
                                                return c, o, p;""")

Removed All!
<Record c=<Node id=22 labels={'Client'} properties={'name': 'Alice', 'billing_address': 'a place', 'shipping_address': 'a place', 'ip': '1.1.1.1'}> o=<Node id=17 labels={'Order'} properties={'name': 'd6', 'id': 'd6', 'deliverdate': '20171212', 'status': 'delivered'}> p=<Node id=12 labels={'Product'} properties={'name': 'kiwi', 'unit': 'box', 'category': 'furit', 'price': 5.3}>>
<Record c=<Node id=22 labels={'Client'} properties={'name': 'Alice', 'billing_address': 'a place', 'shipping_address': 'a place', 'ip': '1.1.1.1'}> o=<Node id=16 labels={'Order'} properties={'name': 'd5', 'id': 'd5', 'deliverdate': '20110801', 'status': 'delivered'}> p=<Node id=13 labels={'Product'} properties={'name': 'cherry', 'unit': 'box', 'category': 'furit', 'price': 4.8}>>
<Record c=<Node id=22 labels={'Client'} properties={'name': 'Alice', 'billing_address': 'a place', 'shipping_address': 'a place', 'ip': '1.1.1.1'}> o=<Node id=15 labels={'Order'} properties={'name': 'd4', 'id': 'd4', 'deliv

In [116]:
graph

In [117]:
with driver.session() as session:
    graph = session.read_transaction(display_all, query="""MATCH (c:Client {name:'Alice'})-[:PLACED]->(o)-[cts:CONTAINS]->(p)
WITH c, o, SUM(cts.quantity * p.price) as order_price
  ORDER BY o.deliverdate
WITH c.name AS name, COLLECT(o) AS os, COLLECT(order_price) as ops
UNWIND [i IN RANGE(0, SIZE(os)-1) |
  {name: name, id: os[i].id, current_order_cost: round(ops[i]),
  other_orders: [x IN os[0..i] + os[i+1..SIZE(os)] | x.id],
  other_orders_costs: [x IN ops[0..i] + ops[i+1..SIZE(os)] | round(x)]
  }] AS result
WITH result.name as name, result.id as order_id, result.current_order_cost as current_order_cost,
result.other_orders as other_orders, result.other_orders_costs as other_orders_costs
UNWIND(other_orders_costs) as unwind_other_orders_costs
return name, order_id, current_order_cost, other_orders, other_orders_costs,
round(stDev(unwind_other_orders_costs)) as other_costs_std;""")

<Record name='Alice' order_id='d4' current_order_cost=790.0 other_orders=['d5', 'd6', 'd1'] other_orders_costs=[48.0, 5.0, 21.0] other_costs_std=22.0>
<Record name='Alice' order_id='d5' current_order_cost=48.0 other_orders=['d4', 'd6', 'd1'] other_orders_costs=[790.0, 5.0, 21.0] other_costs_std=449.0>
<Record name='Alice' order_id='d6' current_order_cost=5.0 other_orders=['d4', 'd5', 'd1'] other_orders_costs=[790.0, 48.0, 21.0] other_costs_std=436.0>
<Record name='Alice' order_id='d1' current_order_cost=21.0 other_orders=['d4', 'd5', 'd6'] other_orders_costs=[790.0, 48.0, 5.0] other_costs_std=441.0>


In [118]:
graph